<a href="https://colab.research.google.com/github/PreetiKumari2208/web_development/blob/main/CONTLO_AI_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 1 | GPT-2 Model & Checkpoints (20 Points)


In [2]:
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    # Implement the Multi-Head Attention module

class PositionwiseFeedforward(nn.Module):
    # Implement the Positionwise Feedforward module

class TransformerLayer(nn.Module):
    # Implement a single Transformer layer with Multi-Head Attention and Feedforward

class GPT2(nn.Module):
    # Implement the GPT-2 model using the Transformer architecture

    def __init__(self):
        super(GPT2, self).__init__()
        # Initialize layers and parameters

    def forward(self, x):
        # Implement the forward pass
        return x

# Sample usage:
model = GPT2()
input_sequence = torch.randint(0, vocab_size, (batch_size, sequence_length))
output = model(input_sequence)



IndentationError: ignored

Task 2 | Transformer Architectural Changes (40 Points)


In [3]:
import torch
import torch.nn as nn

class RotaryPositionalEmbedding(nn.Module):
    # Implement the Rotary Positional Embedding

class GroupQueryAttention(nn.Module):
    # Implement the Group Query Attention mechanism

class SlidingWindowAttention(nn.Module):
    # Implement the Sliding Window Attention mechanism

class GPT2WithAlterations(nn.Module):
    def __init__(self, use_rotary_pos_emb=False, use_group_query_att=False, use_sliding_window_att=False):
        super(GPT2WithAlterations, self).__init__()

        # Original GPT-2 layers (token embedding, transformer layers, etc.)
        self.token_embedding = nn.Embedding(vocab_size, embed_size)
        self.transformer_layers = nn.ModuleList([TransformerLayer() for _ in range(num_layers)])

        # Add alterations based on input flags
        if use_rotary_pos_emb:
            self.rotary_pos_emb = RotaryPositionalEmbedding()
        if use_group_query_att:
            self.group_query_att = GroupQueryAttention()
        if use_sliding_window_att:
            self.sliding_window_att = SlidingWindowAttention()

    def forward(self, x):
        # Original GPT-2 forward pass
        token_embed = self.token_embedding(x)
        for layer in self.transformer_layers:
            token_embed = layer(token_embed)

        # Apply alterations if specified
        if hasattr(self, 'rotary_pos_emb'):
            token_embed = self.rotary_pos_emb(token_embed)
        if hasattr(self, 'group_query_att'):
            token_embed = self.group_query_att(token_embed)
        if hasattr(self, 'sliding_window_att'):
            token_embed = self.sliding_window_att(token_embed)

        return token_embed

# Instantiate the model with desired alterations
model = GPT2WithAlterations(use_rotary_pos_emb=True, use_group_query_att=True, use_sliding_window_att=True)

# Sample usage:
input_sequence = torch.randint(0, vocab_size, (batch_size, sequence_length))
output = model(input_sequence)


IndentationError: ignored

Task 3: Training Loop Implementation (40 Points)


In [ ]:
import torch
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel
from torch.cuda.amp import GradScaler, autocast
from fsdp import FullyShardedDataParallel

# Assuming you have a GPT2 model as GPT2WithAlterations from Task 2
model = GPT2WithAlterations(use_rotary_pos_emb=True, use_group_query_att=True, use_sliding_window_att=True)
model = model.to(device)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Define your DataLoader
# ...

# Training loop for a single GPU
def train_single_gpu(model, dataloader, criterion, optimizer):
    model.train()
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Training loop for DistributedDataParallel (DDP)
def train_ddp(model, dataloader, criterion, optimizer, device, rank, world_size):
    model = DistributedDataParallel(model, device_ids=[rank])
    model = model.to(device)
    model.train()

    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Training loop for Fully Sharded Data Parallel (FSDP)
def train_fsdp(model, dataloader, criterion, optimizer, device):
    fsdp_model = FullyShardedDataParallel(model)
    fsdp_model = fsdp_model.to(device)
    fsdp_model.train()

    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = fsdp_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Example usage
train_single_gpu(model, train_dataloader, criterion, optimizer)
# For DDP
# train_ddp(model, train_dataloader, criterion, optimizer, device, rank, world_size)
# For FSDP
# train_fsdp(model, train_dataloader, criterion, optimizer, device)
